In [1]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [3]:
from numba.typed import List
from numba import jit, njit, vectorize

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [5]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # number of skipped datapoints to start next window
print(sliding_size)

16


# Feature sets

In [6]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_rms(x, axis=None):
    return np.sqrt(np.mean(x ** 2, axis=axis))

def calc_sma_for_window(data):
    return np.sum(data) / len(data)


def calc_sma_adv_for_window(data):
    return np.sum(data - np.mean(data) / len(data))


def calc_absolutes_for_list(list):
    return ([abs(i) for i in list])

def get_sma(data): 
    sma_sim = calc_sma_for_window(data)
    sma_adv = calc_sma_adv_for_window(data)

    sma_sim_abs = calc_sma_for_window(calc_absolutes_for_list(data))
    sma_adv_abs = calc_sma_adv_for_window(calc_absolutes_for_list(data))

    return sma_sim, sma_adv, sma_sim_abs, sma_adv_abs

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [7]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

# Feature extraction and Vector_Normalization on signal

In [8]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    win = np.array(wind)
    
    
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [9]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [10]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
   
    for subject in range(1,18):
        file_path = dataset_path + '\subject{0}_ideal.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns
        X = tmp_db.iloc[:, 0:-1]
        X = np.array(X)
    
        Y = tmp_db.iloc[:, -1]
        Y = np.array(Y)
        data_normalized = preprocessing.normalize(X, norm  = 'l1') # normalization on signal
        X = data_normalized
        X = pd.DataFrame(X)
        Y = pd.DataFrame(Y)
        tmp_db = pd.concat([X, Y], axis=1)
        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [11]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0
    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

In [12]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
    
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    
    return accuracy

In [13]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [14]:
dataset_csv_path = r"D:\projec\proj\data_ideal_null"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [15]:
ALL.head(10)

group         0         1         2         3         4          5  \
0      1 -0.002825  0.000036  0.000036 -0.065652  0.033520 -49.574800   
1      1 -0.002823  0.000036  0.000036 -0.065406  0.033520 -49.536156   
2      1 -0.002822  0.000036  0.000036 -0.065262  0.033512 -49.525826   
3      1 -0.002821  0.000036  0.000036 -0.065262  0.033512 -49.500580   
4      1 -0.002820  0.000037  0.000037 -0.065262  0.033512 -49.488968   
5      1 -0.002819  0.000037  0.000037 -0.065262  0.033512 -49.477226   
6      1 -0.002818  0.000037  0.000037 -0.065064  0.033512 -49.451406   
7      1 -0.002817  0.000037  0.000037 -0.065055  0.033512 -49.429989   
8      1 -0.002815  0.000037  0.000037 -0.065055  0.033512 -49.408759   
9      1 -0.002815  0.000037  0.000037 -0.065055  0.033361 -49.398419   

          6     7     8  ...           233           234       235       236  \
0  7.228819  69.0  77.0  ...  1.475747e-09  2.898192e-09  0.000026  0.000002   
1  7.228819  70.0  77.0  ...  1.335847e-09  2.926322e-09  0.000026  0.000002   
2  7.228819  70.0  81.0  ...  1.284238e-09  2.976147e-09  0.000026  0.000002   
3  7.228819  70.0  78.0  ...  1.200999e-09  3.027762e-09  0.000026  0.000002   
4  7.228819  72.0  75.0  ...  1.263134e-09  2.889104e-09  0.000026  0.000002   
5  7.228819  73.0  73.0  ...  1.313081e-09  3.013093e-09  0.000026  0.000002   
6  7.228819  74.0  72.0  ...  1.285183e-09  2.939979e-09  0.000026  0.000002   
7  7.228819  76.0  73.0  ...  1.475699e-09  2.909299e-09  0.000026  0.000002   
8  7.228819  77.0  73.0  ...  1.415133e-09  2.693412e-09  0.000026  0.000002   
9  7.228819  75.0  71.0  ...  1.502510e-09  2.405752e-09  0.000026  0.000002   

            237           238      239       240       241  242  
0  7.220587e-07  4.515309e-07  0.00002  0.000002  0.000021    0  
1  7.225791e-07  4.649423e-07  0.00002  0.000002  0.000021    0  
2  7.239200e-07  4.722721e-07  0.00002  0.000002  0.000021    0  
3  7.259969e-07  4.776449e-07  0.00002  0.000002  0.000021    0  
4  7.261745e-07  4.850802e-07  0.00002  0.000002  0.000021    0  
5  7.258662e-07  4.936373e-07  0.00002  0.000002  0.000021    0  
6  7.256215e-07  5.031632e-07  0.00002  0.000002  0.000021    0  
7  7.263082e-07  5.132497e-07  0.00002  0.000002  0.000021    0  
8  7.270302e-07  5.240153e-07  0.00002  0.000002  0.000021    0  
9  7.272074e-07  5.359114e-07  0.00002  0.000002  0.000021    0  

[10 rows x 244 columns]

In [16]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec
Model- 1  -  f1 score:  0.8762185850997227
Model- 2  -  f1 score:  0.9524647887323944
Model- 3  -  f1 score:  0.9061111111111111
Model- 4  -  f1 score:  0.9362437864707154
Model- 5  -  f1 score:  0.9175019831254057
Model- 6  -  f1 score:  0.9394180314835427
Model- 7  -  f1 score:  0.9770347855454239
Model- 8  -  f1 score:  0.9361108128422635
Model- 9  -  f1 score:  0.9425788858589793
Model- 10  -  f1 score:  0.9506182320866293
Model- 11  -  f1 score:  0.9033882783882784
Model- 12  -  f1 score:  0.893820416524411
Model- 13  -  f1 score:  0.8720044882583955
Model- 14  -  f1 score:  0.9352852745364736
Model- 15  -  f1 score:  0.9442241607190061
Model- 16  -  f1 score:  0.8166920087852678
Model- 17  -  f1 score:  0.897419036659543


0.9174785097780919

In [17]:
d ={'ALL': [ALL_accuracy]}

In [18]:
df_accuracy = pd.DataFrame(data=d)

In [19]:
df_accuracy

ALL
0  0.917479